In [1]:
import json
import requests
import spotipy
import spotipy.util as util
import pandas as pd
from pandas import DataFrame
import numpy as np
import random
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [3]:
#username可以隨意打
username = '史罷特歪'
scope = 'user-library-read'

token = util.prompt_for_user_token(username, scope,
                          client_id = '使用自己的',
                          client_secret = '使用自己的',
                          # 要在web app 加入 redirect 的白名單
                          redirect_uri = 'http://localhost/')


#client_id = '#client_secret = '                         
# 複製   http://localhost/  後面的東西
#等等出現的對話框 要輸入上面複製的東西



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=&scope=user-library-read in your browser


Enter the URL you were redirected to: ?code=拿到的網址




In [4]:
headers = {"Authorization": "Bearer {}".format(token)}

In [5]:
# https://api.spotify.com/v1/users/你的歌單名字(ex: 中文嘻哈)/playlists/上面歌單/tracks?offset=0&limit=50",headers=headers

#重複抓取所需的資料 get_spotify_data
def get_spotify_data(playlist_uri):
    
    assign_word = "user"
    
    if assign_word in playlist_uri:          #取出網址所需要的部分
        playlist_uri = playlist_uri.split("/")
        get_slash = playlist_uri[6]
        uri_gogo =  "https://api.spotify.com/v1/users/mymusic/playlists/" + get_slash + "tracks?offset=0&limit=50"
    
    else:
        playlist_uri = playlist_uri.split("/")
        get_slash = playlist_uri[4]
        uri_gogo =  "https://api.spotify.com/v1/users/mymusic/playlists/" + get_slash + "tracks?offset=0&limit=50"
    
    
    
    #從API抓資料：
    
    hi = requests.get(uri_gogo,headers = headers)

    
    
    #資料從json轉成串列(每首歌的特徵值為一個字典)(也就是20個字典在一個播放清單的串列裡)：
    data = json.loads(hi.text)         
    someone_songs_attributes = [ ]     # song's attribute
    someone_songs_names = [ ]          #song'name 

    
    def get_song_attributes(response_text):
        return json.loads(response_text)
    
    #拿到歌曲的uri
    for item in data.get('tracks')['items']:
        
        song_ids = item['track']['uri'].split(':')[2]
        song_name = item['track']['name']
        song_attributes = requests.get(f"https://api.spotify.com/v1/audio-features/{song_ids}", headers = headers)
        someone_songs_attributes.append(get_song_attributes(song_attributes.text))
        someone_songs_names.append(song_name)
    
    
    
    #把不需要的變數丟掉(變數第一篩)：    
    for i in someone_songs_attributes:
        i.pop('type')
        i.pop('id')
        i.pop('uri')
        i.pop('track_href')
        i.pop('analysis_url')
        i.pop('time_signature')

    
    
    #將得到的資料整理成表格(取var)：
    df = pd.DataFrame(someone_songs_attributes)
    col_name = df.columns.tolist()
    col_name = sorted(col_name)
    col_name.insert(0, "song")
    df = df.reindex(columns = col_name)
    df["song"] = someone_songs_names
    
    
    
    #把平均或取變異數過後的變數和變數名稱整理成串列：
    column = 0
    variable_title = [ ]
    variable_value = [ ]
    temp = [ ]
    
    for i in col_name:

        if column >= 1:
            
            average_song = df[i].std()
            variable_title.append(i)
            variable_value.append(average_song)
            
        column += 1
    
    
    
    #把要的變數用變數名稱判斷來抓出他的值，加到temp裡面，最後return出這個歌單的變數(變數第二篩)：
    
    for i , j in zip(variable_title,variable_value):
        
        i = str(i)
        j = float(j)
        
        if i == 'speechiness' or i == 'valence':
            temp.append(j)
    
    
    return temp

In [6]:
# path設置你放excel檔的路徑
#/Users/boyen/Desktop/ccClub/spotify專案/資料測試集.xlsx

path = '/Users/boyen/Desktop/ccClub/spotify專案/'
fname = path +'資料測試集.xlsx'

df_test = pd.read_excel(fname, 0)
df_test
#df_test["連結一"]

,連結一,連結二,朋友關係
0,https://open.spotify.com/playlist/2uOx3uPcRy05...,https://open.spotify.com/playlist/5CGCfZ7KHZIa...,1
1,https://open.spotify.com/playlist/7jUXg4DTIc1a...,https://open.spotify.com/playlist/1xFk1UVomG1R...,1
2,https://open.spotify.com/playlist/0lqJgvDkZk3G...,https://open.spotify.com/playlist/6WLvPX3eApl3...,1
3,https://open.spotify.com/playlist/2Cvgv79npINQ...,https://open.spotify.com/playlist/2nlpcmlX0aRj...,1
4,https://open.spotify.com/playlist/4OuBNLWJucVf...,https://open.spotify.com/playlist/40QutHmypjg4...,1
...,...,...,...
94,https://open.spotify.com/playlist/4hMHSNALj4yF...,https://open.spotify.com/playlist/4CgVfYzMtriE...,0
95,https://open.spotify.com/playlist/1HZJcEispDQm...,https://open.spotify.com/playlist/3u0FU6ZCFquA...,0
96,https://open.spotify.com/playlist/5uj7VBiyRrtR...,https://open.spotify.com/playlist/1xRULc1Pl6id...,0
97,https://open.spotify.com/playlist/3S3CQuepi4z9...,https://open.spotify.com/playlist/3HbFyu4KGH8I...,0


In [7]:
values = df_test.values
values = list(values)
#print(values)
print(type(values))

<class 'list'>


In [8]:
print(len(values)) 

99


In [9]:
for i in values:
    print(i)

已刪除

In [17]:
count = 0
calculation = [ ]
ans_big = [ ]          #最後處理過的每組變數，也是最終要的變數串列
ans_set = [ ]          #一組成對樣本計算過後的變數值

for i in range(len(values)):
    
    for j in range(2):
        
        input_url = values[i][j]
        calculation.append(get_spotify_data(input_url))
        
        count += 1                                         
        if count == 2:
            
            for d1 ,d2 in zip(calculation[0],calculation[1]):      #這個迴圈為的是將成對樣本兩兩相減的絕對值存成串列
                
                x = abs(float(d1)-float(d2))      
                ans_set.append(x)               
                
            ans_big.append(ans_set)             
            ans_set = [ ]                        
            count = 0                           
            calculation = [ ]                    

        else:                       #這表示如果是第1個網址，就先不要計算(收割)，先繼續等待第2網址到來才能開搞
            pass
    print(i, 'done')
        
print(ans_big)

0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
[[0.00915075389980011, 0.05728778628014114], [0.006542371663605319, 0.11220211654665962], [0.04310456513842336, 0.05678319082475944], [0.028839062555174058, 0.007555285737287726], [0.019342708530104863, 0.0108623136515

In [18]:
import pickle
file = open(path + '99data_test.pkl', 'wb')
pickle.dump(ans_big, file)
file.close()

In [19]:
pickle_off = open (path + '99data_test.pkl', "rb")
emp = pickle.load(pickle_off)
print(emp)

[[0.00915075389980011, 0.05728778628014114], [0.006542371663605319, 0.11220211654665962], [0.04310456513842336, 0.05678319082475944], [0.028839062555174058, 0.007555285737287726], [0.019342708530104863, 0.010862313651553518], [0.0009649681016766186, 0.10173240498343897], [0.0038012442930115964, 0.11850960027868823], [0.017894200848873425, 0.09618119858351623], [0.0841696765791016, 0.05321945467029532], [0.0003523295887125043, 0.11103139612606275], [0.09222587931838147, 0.04389748688934628], [0.03596572915659746, 0.0463148949770089], [0.08691922921170676, 0.008343967939999808], [0.019674922525278273, 0.012661021710673154], [0.10244937187918188, 0.046046472227672414], [0.05051368349126595, 0.1629043152618277], [0.002404254890708793, 0.009505789558388267], [0.012260867156800234, 0.10508515931523589], [0.08204308488763186, 0.11129161927341152], [2.6027828744371387e-05, 0.02085765677240721], [0.025511540970282857, 0.025178178894163794], [0.006575995970198636, 0.13511544054429334], [0.061556

In [20]:
#將上面的結果轉成兩個lst各自存放
speechiness_lst_zip = [ ]
valence_lst_zip = [ ]

for i in ans_big:
    speechiness_lst_zip.append(i[0])
    valence_lst_zip.append(i[1])
    

In [21]:
#檢查結果
print(speechiness_lst_zip)
print()
print(valence_lst_zip)

[0.00915075389980011, 0.006542371663605319, 0.04310456513842336, 0.028839062555174058, 0.019342708530104863, 0.0009649681016766186, 0.0038012442930115964, 0.017894200848873425, 0.0841696765791016, 0.0003523295887125043, 0.09222587931838147, 0.03596572915659746, 0.08691922921170676, 0.019674922525278273, 0.10244937187918188, 0.05051368349126595, 0.002404254890708793, 0.012260867156800234, 0.08204308488763186, 2.6027828744371387e-05, 0.025511540970282857, 0.006575995970198636, 0.061556777810326785, 0.1047338132089854, 0.09518040991231881, 0.00786820274834747, 0.05257840746121513, 0.040079232267371506, 0.09410109301444454, 0.0385565889083671, 0.014537970882433858, 0.03522223850332319, 0.007806198611753317, 0.02453610606346518, 0.010755243258512197, 0.03708886127893443, 0.08203997746212338, 0.0268340105832042, 0.0866350716676462, 0.03672643503752874, 0.07897487699891127, 0.08427731772960376, 0.046421680925313694, 0.01376523725633512, 0.04962427085672956, 0.10290260793893442, 0.074376854302

In [22]:
relation = [ ]

for i in values:
    relation.append(i[2])

len(relation)

99

In [23]:
#創造dataframe
cross  = list(zip(speechiness_lst_zip, valence_lst_zip, relation))
df_test_final = pd.DataFrame(cross, columns = ["speechiness", "valence", "relationship"])
df_test_final

,speechiness,valence,relationship
0,0.009151,0.057288,1
1,0.006542,0.112202,1
2,0.043105,0.056783,1
3,0.028839,0.007555,1
4,0.019343,0.010862,1
...,...,...,...
94,0.011458,0.042074,0
95,0.006091,0.095504,0
96,0.016703,0.142177,0
97,0.020147,0.112745,0


In [24]:
from sklearn import preprocessing, linear_model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection  import train_test_split
plt.style.use('ggplot')
plt.rcParams['font.family']='SimHei' #⿊體

In [25]:
#切分訓練 測試資料
x = df_test_final[['speechiness','valence']]
y = df_test_final[['relationship']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 20191221)   #random_state 種子值

x_train

,speechiness,valence
80,0.100944,0.060048
43,0.013765,0.162051
62,0.022374,0.037410
73,0.015692,0.013474
28,0.094101,0.159617
...,...,...
90,0.036086,0.034029
89,0.012502,0.122377
25,0.007868,0.048907
15,0.050514,0.162904


In [26]:
from sklearn.preprocessing  import StandardScaler
sc = StandardScaler()
sc.fit(x_train)

x_train_nor=sc.transform(x_train)
x_test_nor=sc.transform(x_test)

In [27]:
#訓練資料分類效果(3個參數)
from sklearn.linear_model  import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train_nor,y_train)

# 印出係數
print(lr.coef_)
# 印出截距
print(lr.intercept_ )

[[0.05915177 0.08194771]]
[0.16945703]


//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [28]:
#檢查樣本外
np.round(lr.predict_proba(x_test_nor),5)

array([[0.48699, 0.51301],
       [0.44135, 0.55865],
       [0.45274, 0.54726],
       [0.46174, 0.53826],
       [0.49437, 0.50563],
       [0.45829, 0.54171],
       [0.4386 , 0.5614 ],
       [0.42884, 0.57116],
       [0.4393 , 0.5607 ],
       [0.46807, 0.53193],
       [0.46693, 0.53307],
       [0.4347 , 0.5653 ],
       [0.44337, 0.55663],
       [0.42591, 0.57409],
       [0.4232 , 0.5768 ],
       [0.44472, 0.55528],
       [0.49693, 0.50307],
       [0.46569, 0.53431],
       [0.43808, 0.56192],
       [0.45508, 0.54492]])

In [43]:
df_test_final = pd.DataFrame(cross, columns = ["speechiness", "valence", "relationship"])

# Removed observations without valence

df_test_final = df_test_final[~df_test_final["speechiness"].isna()]
train, validation = train_test_split(df_test_final, test_size=0.1, random_state = 123)
X_train = train.loc[:, ["speechiness", "valence"]].values
y_train = train.loc[:, "relationship"].values

# Fit Logistic regression classifier

clf = LogisticRegression()
clf.fit(X_train, y_train)
X_validation = validation.loc[:, ["speechiness", "valence"]].values
y_validation = validation.loc[:, "relationship"].values

y_hat = clf.predict(X_validation)
# Calculating confusion matrix

nunique_labels = len(set(y_train))
conf_mat_shape = (nunique_labels, nunique_labels)
conf_mat = np.zeros(conf_mat_shape, dtype=int)

for actual, predict in zip(y_hat, y_validation):
    conf_mat[actual, predict] += 1

# Calculating accuracy

accuracy = (conf_mat[0, 0] + conf_mat[1, 1])/conf_mat.sum()

print(conf_mat)
print("Accuracy: {:.2f}%".format(accuracy * 100))

[[0 0]
 [4 6]]
Accuracy: 60.00%


//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
